In [1]:
%pip install bibtexparser
%pip install vllm
!wget "https://evilscript.eu/upload/files/file.bib"

Note: you may need to restart the kernel to use updated packages.
  Cloning https://github.com/vllm-project/vllm.git to /tmp/pip-req-build-0mqnvxwu
  Running command git clone --filter=blob:none --quiet https://github.com/vllm-project/vllm.git /tmp/pip-req-build-0mqnvxwu
  Resolved https://github.com/vllm-project/vllm.git to commit edb305584bda536ee2b35db9c3e0615b43a5ee02
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Note: you may need to restart the kernel to use updated packages.
--2023-11-18 15:19:27--  https://evilscript.eu/upload/files/file.bib
Resolving evilscript.eu (evilscript.eu)... 89.46.105.45
Connecting to evilscript.eu (evilscript.eu)|89.46.105.45|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.evilscript.eu/upload/files/file.bib [following]
--2023-11-18 15:19:28--  https://www.evilscript.eu/upload/files/file.bib
Resolving www.evil

In [3]:
import pickle

import bibtexparser
from vllm import LLM


def create_prompts(file: str) -> dict:
    with open(file) as bibtex_file:
        bib_database = bibtexparser.load(bibtex_file)

    prompts = {
        "Given the following paper, respond 'yes' if the paper is strictly about AI and its use in Video games/Serious games, 'no' otherwise. Don't write anything else.\n"
        "Paper: {}\n".format(entry['title']) +
        ('Abstract: ' + entry['abstract'] + '\n' if 'abstract' in entry else '') +
        ('Keywords: ' + entry['keywords'] + '\n' if 'keywords' in entry else '') +
        "Response: ": entry
        for entry in bib_database.entries
    }

    return prompts


def save_bibtex(outputs: list, prompts: dict, output_file_name: str) -> None:
    """
    Append the bibtex entry if the output is yes to the output file
    """
    good_bibtex = []

    for output in outputs:
        prompt = output.prompt
        bibtex = prompts[prompt]
        generated_text = output.outputs[0].text.lower()
        # print(f"Prompt: {prompt} -- Generated text: {generated_text}")
        # check the type of the generated text: if it's not a str then it's an error
        if not isinstance(generated_text, str):
            continue
        if "yes" in generated_text:
            # Append the bibtex entry to the output file
            good_bibtex.append(bibtex)

    db = bibtexparser.bibdatabase.BibDatabase()
    db.entries = good_bibtex
    writer = bibtexparser.bwriter.BibTexWriter()
    with open(output_file_name, "w") as f:
        f.write(writer.write(db))


if __name__ == "__main__":
    prompts = create_prompts("file.bib")
    # If there's a pickle file with the outputs, load it
    try:
        with open("outputs.pkl", "rb") as f:
            outputs = pickle.load(f)
    except FileNotFoundError:
        # Create a new LLM object
        llm = LLM(model="mosaicml/mpt-30b-instruct", trust_remote_code=True)
        outputs = llm.generate([prompt for prompt in prompts.keys()])
        # Save outputs to local file using pickle
        with open("outputs.pkl", "wb") as f:
            pickle.dump(outputs, f)
    save_bibtex(outputs, prompts, "output.bib")

INFO 11-19 11:39:06 llm_engine.py:72] Initializing an LLM engine with config: model='mosaicml/mpt-30b-instruct', tokenizer='mosaicml/mpt-30b-instruct', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=auto, tensor_parallel_size=1, quantization=None, seed=0)


/home/paperspace/.cache/huggingface/modules/transformers_modules/mosaicml/mpt-30b-instruct/2e70654433d77f7f6e76bcbce85945ff77646c5d/configuration_mpt.py:97: UserWarning: alibi is turned on, setting `learned_pos_emb` to `False.`
  warnings.warn(f'alibi is turned on, setting `learned_pos_emb` to `False.`')
/home/paperspace/miniforge3/envs/semagram/lib/python3.10/site-packages/vllm/transformers_utils/configs/mpt.py:152: UserWarning: alibi is turned on, setting `learned_pos_emb` to False`
  warnings.warn(


INFO 11-19 11:40:33 llm_engine.py:207] # GPU blocks: 699, # CPU blocks: 195


Processed prompts: 100%|███████████████████████████████████████████████████████| 5260/5260 [09:17<00:00,  9.44it/s]
